In [5]:
import os
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

In [6]:
dataroot = Path("./assets/preprocessed")
#train_path = dataroot / "Train_nus-wide.ndjson"
test_path = dataroot / "Test_nus-wide.ndjson"

In [7]:
test = pl.read_ndjson(test_path)
tsample = test
#tsample = tsample.filter(pl.col("tags").list.len() > 3)
tsample = tsample.sample(4, seed=2)
tsample.glimpse()

Rows: 4
Columns: 4
$ file_name        <str> '0431_2485459033.jpg', '0590_490134926.jpg', '0222_2573539930.jpg', '0394_423363936.jpg'
$ labels     <list[str]> ['sky', 'water', 'clouds', 'architecture', 'city', 'travel', 'street', 'urban', 'europe', 'france', 'window', 'square', 'rain', 'paris', 'cars', 'tourism', 'traffic', 'highway', 'ancient', 'rainy'], ['nature', 'green', 'explore', 'trees', 'grass', 'bench', 'woods', 'wales'], ['sky', 'blue', 'canada', 'building', 'glass', 'toronto', 'office'], ['night', 'desert', 'sign', 'cafe', 'neon']
$ tags       <list[str]> ['cars', 'clouds', 'sky', 'street', 'water', 'window'], ['grass'], ['sky'], ['sign']
$ image_root       <str> './assets/nus-wide/images', './assets/nus-wide/images', './assets/nus-wide/images', './assets/nus-wide/images'



In [8]:
#from scripts.test.compose import trainer, model, data
#os.environ["SPECIFIC_IMGS"] = ",".join(tsample["file_name"])
#
#alignment = {"verticalalignment": "center", "horizontalalignment": "center"}
#
#preds = trainer.predict(model, datamodule=data)
#fig, axs = plt.subplots(ncols=5, nrows=4, figsize=(5.5, 3.5), layout="constrained")
#for ax in axs.ravel():
#    ax.set_axis_off()
#for i, txt in enumerate(["Image", "Truth", "Top 5 pred", "Model pred"]):
#    axs[i,0].text(0, 0.5, txt, **alignment)
#for i, path in enumerate(tsample["file_name"]):
#    img_path = Path(tsample["image_root"][i], path)
#    img = plt.imread(img_path)
#    axs[0,i+1].imshow(img)
#    axs[1,i+1].text(0.5, 0.5, "\n".join(tsample["tags"][i]), **alignment)
#    axs[2,i+1].text(0.5, 0.5, "\n".join(tsample["labels"][i][:5]), **alignment)
#    axs[3,i+1].text(0.5, 0.5, "\n".join(preds[0][i] if len(preds[0][i]) > 0 else ["-"]), **alignment)